# 🚑 Pipeline ELT: Integração de Dados do SAMU (2023-2025)

### 📋 Sobre o Projeto
Este notebook implementa um pipeline de dados seguindo a arquitetura **ELT (Extract, Load, Transform)** para integrar e analisar dados de atendimentos do SAMU. O objetivo é construir um **Data Warehouse** (Esquema Estrela) para responder a perguntas de negócio sobre a eficiência e demanda do serviço.

### 🛠️ Tecnologias e Justificativas
Para a execução deste projeto, selecionamos uma *stack* tecnológica enxuta e eficiente:

* **Python (Pandas):**
    * *Por que?* O Pandas oferece funções robustas (`read_csv`, `to_sql`) para lidar com a leitura de arquivos CSV com sujeiras (como o arquivo de 2025 que veio sem cabeçalho) e gerenciar a conexão com o banco de dados.

* **SQLite3:** Utilizado como motor de **Banco de Dados (Data Warehouse Local)**.
    * *Por que?* Por ser um banco de dados *serverless* (baseado em arquivo), ele elimina a necessidade de configurar servidores complexos, sendo ideal para prototipagem rápida e garantindo que todo o processamento ocorra em ambiente SQL nativo.

* **SQL:** Utilizado para **Transformação, Limpeza e Modelagem**.
    * *Por que?* Em uma abordagem ELT, delegar o processamento pesado para o banco de dados é mais performático do que iterar linhas em memória. Usamos SQL para normalizar textos (`UPPER`), tratar nulos (`COALESCE`) e realizar *Joins* complexos para criar as Tabelas Fato e Dimensão.

* **Matplotlib & Seaborn:** Utilizados para **Analytics e Visualização**.
    * *Por que?* Bibliotecas padrão de mercado para geração de gráficos estáticos de alta qualidade, essenciais para compor o relatório final.

In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import os

## Setup - Criação da Estrutura de Arquivos

In [2]:
# Configuração estética dos gráficos
sns.set_theme(style="whitegrid")
sns.set_context("talk")

In [3]:
# Função auxiliar para gravar os scripts SQL no disco.
# Isso garante que a lógica de transformação fique separada da lógica de orquestração (Python).
def criar_arquivo_sql(caminho, conteudo):
    os.makedirs(os.path.dirname(caminho), exist_ok=True)
    with open(caminho, 'w') as f:
        f.write(conteudo)
    print(f"📁 Script SQL criado: {caminho}")

In [4]:
# Tenta fechar a conexão antiga se ela existir na memória
try:
    conn.close()
except:
    pass # Se não existir conexão aberta, segue o jogo

# Agora tenta remover o arquivo
db_path = 'samu_dw.db'
if os.path.exists(db_path):
    try:
        os.remove(db_path)
        print(f"♻️ Banco antigo '{db_path}' removido com sucesso.")
    except PermissionError:
        print(f"⚠️ AVISO: Não foi possível apagar '{db_path}' porque ele está em uso.")
        print("   -> O código vai reutilizar o banco existente (sem recriar do zero).")
        print("   -> Dica: Para recriar do zero, Reinicie o Kernel do notebook.")

# Cria a nova conexão
conn = sqlite3.connect(db_path)
print("✅ Conexão estabelecida.")

# --- 2. DEFINIÇÃO DE SCHEMA (Necessário para o arquivo sem cabeçalho) ---
colunas_padrao = [
    '_id', 'data', 'hora_minuto', 'municipio', 'bairro', 'endereco',
    'origem_chamado', 'tipo', 'subtipo', 'sexo', 'idade',
    'motivo_finalizacao', 'motivo_desfecho'
]

♻️ Banco antigo 'samu_dw.db' removido com sucesso.
✅ Conexão estabelecida.


DEFINIÇÃO DOS SCRIPTS SQL (LÓGICA DE NEGÓCIO)

In [5]:
# Staging: Seleciona colunas úteis e padroniza textos (UPPER) para evitar duplicatas.

sql_stg = """
DROP TABLE IF EXISTS stg_samu;
CREATE TABLE stg_samu AS
SELECT
    _id as id_atendimento,
    ano_origem,
    data as data_ocorrencia,
    hora_minuto,
    UPPER(municipio) as municipio, -- Normalização: 'Recife' vira 'RECIFE'
    UPPER(bairro) as bairro,
    UPPER(tipo) as tipo_ocorrencia,
    UPPER(COALESCE(subtipo, 'NAO INFORMADO')) as subtipo, -- Tratamento de Nulos
    motivo_desfecho,
    idade,
    sexo
FROM stg_samu_raw;
"""

In [6]:
# Dimensão Local: Cria tabela única de locais para o Esquema Estrela.
sql_dim_local = """
DROP TABLE IF EXISTS dim_local;
CREATE TABLE dim_local AS
SELECT DISTINCT 
    municipio, 
    bairro
FROM stg_samu
WHERE municipio IS NOT NULL;
"""

In [7]:
# Dimensão Tempo: Explode a data em atributos analíticos (Ano, Mês, Dia).

sql_dim_tempo = """
DROP TABLE IF EXISTS dim_tempo;
CREATE TABLE dim_tempo AS
SELECT DISTINCT
    data_ocorrencia as id_tempo,
    strftime('%Y', data_ocorrencia) as ano,
    strftime('%m', data_ocorrencia) as mes,
    strftime('%d', data_ocorrencia) as dia,
    strftime('%w', data_ocorrencia) as dia_semana
FROM stg_samu
WHERE data_ocorrencia IS NOT NULL;
"""

In [8]:
# Dimensão Motivo: Limpeza pesada de strings.
# Removemos números (ex: "1. SUCESSO") para padronizar o motivo.

sql_dim_motivo = """
DROP TABLE IF EXISTS dim_motivo;
CREATE TABLE dim_motivo AS
SELECT DISTINCT
    tipo_ocorrencia as tipo,
    subtipo,
    CASE 
        WHEN motivo_desfecho LIKE '%1. %' THEN SUBSTR(motivo_desfecho, 4)
        WHEN motivo_desfecho LIKE '%2. %' THEN SUBSTR(motivo_desfecho, 4)
        WHEN motivo_desfecho LIKE '%3. %' THEN SUBSTR(motivo_desfecho, 4)
        ELSE UPPER(motivo_desfecho)
    END as motivo_desfecho_limpo
FROM stg_samu;
"""

In [9]:
# Tabela Fato: O coração do Data Warehouse.
# Usa CTEs (Common Table Expressions) para organizar os joins.

sql_fato = """
-- Garante limpeza prévia
DROP TABLE IF EXISTS fato_atendimentos;

-- Define estrutura tipada (Boa prática de Engenharia)
CREATE TABLE fato_atendimentos (
    id_atendimento TEXT,
    ano_origem INTEGER,
    id_tempo TEXT,
    id_local INTEGER,
    id_motivo INTEGER,
    idade INTEGER,
    sexo TEXT,
    hora_minuto TEXT
);

-- Insere dados transformados
INSERT INTO fato_atendimentos
WITH 
source   AS ( SELECT * FROM stg_samu ),
d_local  AS ( SELECT rowid as id_local, municipio, bairro FROM dim_local ),
d_motivo AS ( SELECT rowid as id_motivo, tipo, subtipo, motivo_desfecho_limpo FROM dim_motivo ),
d_tempo  AS ( SELECT id_tempo FROM dim_tempo )

SELECT 
    base.id_atendimento,
    base.ano_origem,
    t.id_tempo,
    l.id_local,
    m.id_motivo,
    base.idade,
    base.sexo,
    base.hora_minuto
FROM source AS base
-- Join por texto normalizado (garante match mesmo com diferenças de caixa)
LEFT JOIN d_local l ON base.municipio = l.municipio AND base.bairro = l.bairro
LEFT JOIN d_motivo m 
    ON base.tipo_ocorrencia = m.tipo 
    AND base.subtipo = m.subtipo
    AND ((base.motivo_desfecho LIKE '%1. %' AND m.motivo_desfecho_limpo = SUBSTR(base.motivo_desfecho, 4)) 
         OR (m.motivo_desfecho_limpo = UPPER(base.motivo_desfecho)))
LEFT JOIN d_tempo t ON base.data_ocorrencia = t.id_tempo;
"""

In [10]:
# Criação física dos arquivos na pasta sql/
criar_arquivo_sql('../sql/staging/stg_samu.sql', sql_stg)
criar_arquivo_sql('../sql/marts/dim_local.sql', sql_dim_local)
criar_arquivo_sql('../sql/marts/dim_tempo.sql', sql_dim_tempo)
criar_arquivo_sql('../sql/marts/dim_motivo.sql', sql_dim_motivo)
criar_arquivo_sql('../sql/marts/fato_atendimentos.sql', sql_fato)

📁 Script SQL criado: ../sql/staging/stg_samu.sql
📁 Script SQL criado: ../sql/marts/dim_local.sql
📁 Script SQL criado: ../sql/marts/dim_tempo.sql
📁 Script SQL criado: ../sql/marts/dim_motivo.sql
📁 Script SQL criado: ../sql/marts/fato_atendimentos.sql


### Etapa 1: Extract & Load (EL)

Nesta etapa, ingerimos os arquivos CSV. 
Foi identificado um **desafio de qualidade de dados**: o arquivo de **2025 não possui cabeçalho**, enquanto os de 2023 e 2024 possuem.

**Estratégia:**
1.  Definimos manualmente os nomes das colunas para garantir alinhamento (Schema Enforcement).
2.  Adicionamos a coluna `ano_origem` para rastrear de qual arquivo o dado veio.
3.  Carregamos tudo para uma tabela de *Staging* (`stg_samu_raw`) no banco, sem filtros, preservando o dado original.

In [11]:
try:
    # Leitura dos CSVs (Extract)
    # 2023 e 2024 possuem cabeçalho, o Pandas lê automaticamente
    df_23 = pd.read_csv('../data/samu_2023.csv')
    df_24 = pd.read_csv('../data/samu_2024.csv')
    
    # 2025 NÃO tem cabeçalho. Usamos 'header=None' e passamos os nomes manualmente
    df_25 = pd.read_csv('../data/samu_2025.csv', header=None, names=colunas_padrao)

    # Enriquecimento (Linhagem do dado)
    df_23['ano_origem'] = 2023
    df_24['ano_origem'] = 2024
    df_25['ano_origem'] = 2025

    # Consolidação e Carga (Load)
    # Concatenamos os 3 anos em um único DataFrame
    df_total = pd.concat([df_23, df_24, df_25])
    
    # Enviamos para o SQLite. 
    # if_exists='replace': recria a tabela se ela já existir.
    # index=False: não queremos salvar o índice numérico do Pandas.
    df_total.to_sql('stg_samu_raw', conn, if_exists='replace', index=False)
    
    print(f"✅ Sucesso! {len(df_total)} registros carregados na tabela 'stg_samu_raw'.")

except Exception as e:
    print(f"❌ Erro crítico na extração: {e}")

C:\Users\sofia\AppData\Local\Temp\ipykernel_34452\505856528.py:8: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_25 = pd.read_csv('../data/samu_2025.csv', header=None, names=colunas_padrao)


✅ Sucesso! 539519 registros carregados na tabela 'stg_samu_raw'.


### Etapa 2: Transformação - SQL

Aqui ocorre a transformação. Utilizamos SQL para criar o **Esquema Estrela**.
O objetivo é normalizar os dados para evitar repetição de strings (como nomes de bairros repetidos milhares de vezes) e padronizar inconsistências.

**Tabelas Criadas:**
1.  **Dimensões (`dim_`):** Tabelas auxiliares que contêm os atributos descritivos (Onde? Quando? Por quê?).
2.  **Fato (`fato_`):** Tabela central que contém as métricas e as chaves estrangeiras (IDs) apontando para as dimensões.

In [12]:
def executar_script_sql(caminho, conn):
    print(f"🔄 Executando modelo: {os.path.basename(caminho)}...")
    try:
        with open(caminho, 'r') as f:
            query = f.read()
        
        # 'executescript' permite rodar múltiplos comandos (DROP + CREATE + INSERT)
        conn.executescript(query)
        print(f"   ✅ Sucesso.")
    except Exception as e:
        print(f"   ❌ Erro: {e}")

In [13]:
# 1. Staging (Limpeza Inicial)
executar_script_sql('../sql/staging/stg_samu.sql', conn)

🔄 Executando modelo: stg_samu.sql...
   ✅ Sucesso.


In [14]:
# 2. Dimensões (Devem ser criadas antes da Fato)
executar_script_sql('../sql/marts/dim_local.sql', conn)
executar_script_sql('../sql/marts/dim_tempo.sql', conn)
executar_script_sql('../sql/marts/dim_motivo.sql', conn)

🔄 Executando modelo: dim_local.sql...
   ✅ Sucesso.
🔄 Executando modelo: dim_tempo.sql...
   ✅ Sucesso.
🔄 Executando modelo: dim_motivo.sql...
   ✅ Sucesso.


In [15]:
# 3. Fato (Tabela final que une tudo)
executar_script_sql('../sql/marts/fato_atendimentos.sql', conn)

🔄 Executando modelo: fato_atendimentos.sql...
   ✅ Sucesso.


### Etapa 3: Sanity Check (Qualidade de Dados)

Após a carga, é crucial verificar se os dados estão íntegros.
Identificamos uma sujeira: **linhas de cabeçalho do CSV original foram importadas como dados**.

Isso acontece porque, ao juntar múltiplos arquivos, se não tratarmos corretamente, o cabeçalho do segundo arquivo vira uma linha de registro no meio da tabela.
Abaixo, detectamos esses registros (onde o ID é a string `_id`) e aplicamos uma limpeza via `DELETE`.

In [16]:
# --- Sanity Check ---
print("\n🔍 Verificando integridade dos dados...")
lixo = conn.execute("SELECT COUNT(*) FROM fato_atendimentos WHERE id_atendimento = '_id'").fetchone()[0]

if lixo > 0:
    print(f"⚠️ ALERTA: {lixo} registros sujos detectados (cabeçalho duplicado).")
    conn.execute("DELETE FROM fato_atendimentos WHERE id_atendimento = '_id'")
    conn.commit()
    print("🧹 Limpeza automática realizada com sucesso.")
else:
    print("✅ Dados íntegros. Nenhuma anomalia detectada.")


🔍 Verificando integridade dos dados...
⚠️ ALERTA: 1 registros sujos detectados (cabeçalho duplicado).
🧹 Limpeza automática realizada com sucesso.
